In [11]:
import numpy as np
import pandas as pd

# CONSTANTS
n = 0          
g = 20        # Degrees of freedom
c1 = np.sqrt(4*(np.pi**3)/45)
c2 = 2*(np.pi**2)/45
DecayConstant = 1
M_Pl = 1.2206e22  # Planck mass in MeV

In [153]:
# COUPLING CONSTANTS
k_SC, k_SV = 1e-6,1e-7 #Talk to Evan and change these numbers
k_Point = 1e-3


# PARTICLE VELOCITIES
V_Pion = 54 #m/s 
V_Kaon = 54
V_Baryon = 54

# Particle masses in MeV
m = 0.511e-3      # Electron mass in GeV
M_Pl = 1.2206e19  # Planck mass in GeV
M_Proton, M_Neutron, M_Lambda = 0.9382720813, 0.93957, 1.115683
M_ChargedPion, M_NeutralPion = 0.13957039, 0.1349768
M_ChargedKaon, M_NeutralKaon = 0.493677, 0.497611

#Testing pion masses to measure how mass effects these calcs
M_NeutralPion = 1e-1
M_NeutralKaon = 4e-1

Test_baryon = 1e-1 #MeV
Test_meson = 1e0
k_test = 1e-2

In [146]:
# Load the data from CSV files
Gstar_df = pd.read_csv("Gstar.csv")
GstarS_df = pd.read_csv("GstarS.csv")

Gstar = dict(zip(Gstar_df.iloc[:, 0], Gstar_df.iloc[:, 1]))
GstarS = dict(zip(GstarS_df.iloc[:, 0], GstarS_df.iloc[:, 1]))

def g_star(Temp):
    # Find closest temperature value in Gstar
    closest_temp = min(Gstar.keys(), key=lambda t: abs(t - Temp))
    return Gstar[closest_temp]

def g_starS(Temp):
    # Find closest temperature value in GstarS
    closest_temp = min(GstarS.keys(), key=lambda t: abs(t - Temp))
    return GstarS[closest_temp]

In [2]:
def Y_EQ_GT3(x, particle): #x >> 3
    mass = particle_mass.get(particle, 1) # Get particle masses, default to 1 if not found
    #print('mass/x = ', mass/x)
    #print('mass = ', mass)
    #print('x = ', x)
    #print('g = ', g_starS(mass/x))
    #print('Y_EQ =' , 0.145 * (g / g_starS(mass / x)) * x**(3/2) * np.exp(-x))
    return 0.145 * (g / g_starS(mass / x)) * x**(3/2) * np.exp(-x) #eqn 5.25

def Y_EQ_LT3(x, particle): #x << 3
    mass = particle_mass.get(particle, 1) # Get particle masses, default to 1 if not found
    #print('mass/x = ', mass/x)
    #print('mass = ', mass)
    #print('x = ', x)
    #print('g = ', g_starS(mass/x))
    #print('Y_EQ =' , 0.145 * (g / g_starS(mass / x)) * x**(3/2) * np.exp(-x))
    return 0.278 * (g / g_starS(mass / x)) #eqn 5.25

def n_eq(x, particle):
    mass = particle_mass.get(particle, 1)
    T = mass/x
    return (g/(2*(np.pi**2)))*(mass**2)*(T)*kn(2, x)

def Y_EQ(x, particle):
    mass = particle_mass.get(particle, 1)
    return n_eq(x, particle) / s(x, particle)

# Define H(m) function (Hubble parameter)
def H_m(x, particle):
    mass = particle_mass.get(particle, 1)
    #print('H_m = ', (c1 * np.sqrt(g_star(mass / x)) * (mass / x)**2) / M_Pl)
    return (c1 * np.sqrt(g_star(mass / x)) * (mass / x)**2) / M_Pl

# Define s(x, m) function (entropy density)
def s(x, particle):
    mass = particle_mass.get(particle, 1)
    #print('s = ', c2 * g_star(mass / x) * (mass / x)**3)
    return c2 * g_starS(mass / x) * (mass / x)**3

In [63]:
def Energy(Velocity, Mass): #E = sqrt(m^2 + p^2)
    P = 0.5*Mass*(Velocity**2)
    return np.sqrt(Mass**2 + P**2) #energy will change with the temperature. the cross section is (1/2s)(amplitude)

# Cross-section formula for a single Lambda
def sigma(s, *lambdas, particle=None):
    if particle is not None:
        lambdas = particle
    return sum(Lambda / (2 * s) for Lambda in lambdas) #Make an array that has lambdas_pion, lambdas_proton, lambdas_neutron, etc. for all of their interactions
    
def thermally_averaged_cross_section(T, m, Lambda):
    def integrand(s):
        return sigma(s, Lambda) * np.sqrt(s) * kn(1, np.sqrt(s) / T)
        
    integral_result, error = quad(integrand, 4 * m**2, np.inf)
    return (1 / (8 * m**4 * T * kn(2, m / T))) * integral_result

# Lambda_x function for freeze-out dynamics
def lambda_x(x, g_s, g_star, Lambda, particle):
    H_m_value = H_m(x, particle)  
    mass = particle_mass.get(particle, 1)
    T = mass / x
    thermally_avg_sigma = thermally_averaged_cross_section(T, mass, Lambda)
    s_value = s(x, particle)
    return (x * thermally_avg_sigma * s_value) / H_m_value # Equation 1.57 in my thesis notes.


#ANOTHER VARIATION OF LAMBDA_X
"""
def lambda_x(x, Lambda, particle):
    mass = particle_mass.get(particle, 1)
    T = mass / x
    thermally_avg_sigma = thermally_averaged_cross_section(T, mass, Lambda)
    return 0.264*(g_starS(mass/x)/(np.sqrt(g_star(mass/x))))*M_Pl*mass*thermally_avg_sigma
""" 

# Differential equation for relic abundance
def dYdx(x, Y, Lambda, particle):
    ratio = particle_ratios.get(particle, 1)  # Get the particle's ratio; default to 1 if not found

    x_ratio = x*ratio
    """
    if x <= 3:
        Y_eq = Y_EQ_LT3(x_ratio, particle)
    else:
        Y_eq = Y_EQ_GT3(x_ratio, particle)
    """
    Y_eq = Y_EQ(x, particle)
    lambda_val = lambda_x(x_ratio, g_starS, g_star, Lambda, particle)

    if x > 100:  # Arbitrary cutoff for equilibrium
        return np.array([0])
    result = -lambda_val * x_ratio**(-n-2) * (Y**2 - Y_eq**2)
    
    return np.asarray([result])

def get_variable_name(var, global_vars):
    names = [name for name, value in global_vars.items() if value is var]
    return names[0] if names else "unknown"

In [19]:
def AvgDecayRate(x):
    return np.exp(-DecayConstant*x) #Ae^-lambda*t but using x instead

def CoupleddYdx(x, Y, Y_prod, AvgDecayRate, Lambda):
    Y_eq = Y_EQ(x)  
    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)
    AvgDecayRate_val = AvgDecayRate(x)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
    #NOTE THAT YOU NEED TO DEFINE A DECAY RATE, Y_PROD AND Y_PROD_EQ
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2) - ((1.661*x)/((m**2)*(g_star**1/2))*AvgDecayRate_val*(Y-Y_eq*(Y_prod/Y_prod_eq)**2)) 
    return np.array([result])

def dYproddx(x, Y, Y_prod, AvgDecayRate, Lambda):
    AvgDecayRate_val = AvgDecayRate(x)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
        
    result = - ((1.661*x)/((m**2)*(g_star**1/2))*AvgDecayRate_val*(Y-Y_eq*(Y_prod/Y_prod_eq)**2))
    return np.array([result])

# To do list
##### Want sum(m_P * Y) = 4e-10 GeV so all the particle masses multiplied by the relic abundance should be equal to 4e-10 GeV

In [22]:
#def sigma(s, Lambda):
#    return Lambda / (2 * s)

"""
# Differential equation for relic abundance
def dYdx(x, Y, Lambda):
    Y_eq = Y_EQ(x)  
    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)
    
    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed
        return np.array([0])
    
    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2)
    return np.array([result])
"""

'\n# Differential equation for relic abundance\ndef dYdx(x, Y, Lambda):\n    Y_eq = Y_EQ(x)  \n    lambda_val = lambda_x(x, m, g_starS, g_star, Lambda)\n    \n    if x > 100:  # Arbitrary cutoff for equilibrium; modify as needed\n        return np.array([0])\n    \n    result = -lambda_val * x**(-n-2) * (Y**2 - Y_eq**2)\n    return np.array([result])\n'